In [1]:
from lib import SyncedProp

import ipywidgets as widgets

caption = widgets.Label(value='The values of slider1 and slider2 are synchronized')
slider1, slider2 = widgets.IntSlider(description='Slider 1'),\
                    widgets.IntSlider(description='Slider 2')
display(caption, slider1, slider2)

Label(value='The values of slider1 and slider2 are synchronized')

IntSlider(value=0, description='Slider 1')

IntSlider(value=0, description='Slider 2')

In [2]:
p = SyncedProp()
p.sync_prop(slider1, 'value')
p.sync_prop(slider1, 'value')
slider2.value = 87
p.sync_prop(slider2, 'value')
assert p.value == 87
p.value = 45
assert slider1.value == slider2.value == 45

In [3]:
p2 = SyncedProp((slider1, 'value'), (slider2, 'value'))
assert p2.value == p.value

In [4]:
def out_content(value):
    print(f"Value of the slider: {value}")
out = widgets.interactive_output(out_content, {'value': p})
display(out)

Output()

In [5]:
from lib import ComputedProp

slider3 = widgets.IntSlider(description='Slider 3')
checkbox = widgets.Checkbox(description='Sum is greater than 50', disabled=True)

sum_greater_than_100 = ComputedProp((slider1, 'value', 'v1'), (slider3, 'value', 'v3'), f=lambda v1,v3:v1+v3>50)

# SyncedProp((checkbox, 'value')).add_input_prop(sum_greater_than_100, 'value')
SyncedProp(checkbox).add_input_prop(sum_greater_than_100)
display(slider3, checkbox)

IntSlider(value=0, description='Slider 3')

Checkbox(value=False, description='Sum is greater than 50', disabled=True)

In [6]:
# dynamically update the name of slider 3
dropdown = widgets.Dropdown(
    options=['Slider 3', 'Some Fancy Slider', 'Stupid Slider', 'Random Slider'],
    description='Set name of Slider 3'
)

SyncedProp().add_output_prop(slider3, 'description').add_input_prop(dropdown, 'value')
display(dropdown)

Dropdown(description='Set name of Slider 3', options=('Slider 3', 'Some Fancy Slider', 'Stupid Slider', 'Rando…

In [7]:
# computed prop with unnamed inputs, default prop name to 'value'
from datetime import datetime
def get_time():
    return str(datetime.now()) 

# only specifying a 2-tuple to ComputedProp
incrementing_num = ComputedProp((slider3, 'description'), f=get_time)
t = widgets.Text()
# default to 'value'
SyncedProp().add_output_prop(t).add_input_prop(incrementing_num)
display(t)

Text(value='2022-03-01 11:59:06.419193')

## Sync

In [8]:
from lib import Prop

in1 = Prop(1)
assert in1.value == 1
in2 = Prop(2)
assert in2.value == 2

# input sync option specifies whether to update immediately based on the input prop
s_in = SyncedProp().add_input_prop(in1, sync=True).add_input_prop(in2, sync=False)
assert in1.value == 1
assert in2.value == 2
assert s_in.value == 1 # not updated by in2

in2.value = 22
assert s_in.value == 22 # update is triggered by subsequent changes

# output sync option specifies whether to update the component immediately based on current value
out1 = Prop(None)
out2 = Prop(None)

s_out = SyncedProp()
s_out.value = 42
s_out.add_output_prop(out1, sync=True).add_output_prop(out2, sync=False)

assert out1.value == 42
assert out2.value == None # out2's value is not updated immediately
s_out.value = 17
assert out1.value == out2.value == s_out.value == 17 # only updated after subsequent changes

## "Maybe" in `ComputedProp`

In [9]:
# Thought: Is it possible to add types to prop?
in1 = Prop(1)
in2 = Prop(1)
in1_plus_in2 = ComputedProp() \
    .add_inputs((in1, 'value', 'lhs'), (in2, 'value', 'rhs')) \
    .set_output(lambda lhs, rhs: lhs+rhs) \
    .resync()
in1_plus_in2.value

2

In [15]:
in1.value = None
# lambda won't explode
assert in1_plus_in2.value is None
in1.value = 4
in1_plus_in2.value

None value detected in inputs of ComputedProp({'lhs': None, 'rhs': 1})


5